# Kafka Producer

In [1]:
import joblib
import pandas as pd

# Loading the Model

In [2]:
loaded_model=joblib.load('../Model/model_4')

In [3]:
df=pd.read_csv('third_result.csv')

In [4]:
x=df.drop(['Date','Time','Anomaly_Score','Anomaly'],axis=1)

In [5]:
x

,Host,Method,Endpoint,Protocol,Status Code,Content Size,No of Requests
0,2415145644,4,236,6,6,248.0,62
1,2757421940,4,10,6,0,5952.0,5
2,1123632510,4,400,6,7,146.0,20
3,1123632510,4,177,6,0,491.0,20
4,3118324652,4,40,6,7,548.0,1
...,...,...,...,...,...,...,...
5731,2584535982,7,240,6,7,146.0,6
5732,2584535982,4,240,6,7,146.0,6
5733,2584535982,7,10,6,8,150.0,6
5734,2584535982,7,240,6,7,146.0,6


In [6]:
y=df.Anomaly

In [7]:
y

0       0
1       0
2       0
3       0
4       0
       ..
5731    0
5732    0
5733    0
5734    0
5735    0
Name: Anomaly, Length: 5736, dtype: int64

# Connecting to the server and Creating topic for Kafka

In [8]:
import json
from kafka.admin import NewTopic,KafkaAdminClient
from kafka import KafkaProducer
from time import sleep

In [9]:
topic=NewTopic(name='Topic3',num_partitions=3,replication_factor=1)

In [10]:
admin=KafkaAdminClient(bootstrap_servers='localhost:9092')

In [11]:
admin.create_topics([topic])

TopicAlreadyExistsError: [Error 36] TopicAlreadyExistsError: Request 'CreateTopicsRequest_v3(create_topic_requests=[(topic='Topic3', num_partitions=3, replication_factor=1, replica_assignment=[], configs=[])], timeout=30000, validate_only=False)' failed with response 'CreateTopicsResponse_v3(throttle_time_ms=0, topic_errors=[(topic='Topic3', error_code=36, error_message="Topic 'Topic3' already exists.")])'.

In [14]:
def json_serializer(data):
    return json.dumps(data).encode('utf-8')

In [15]:
producer=KafkaProducer(bootstrap_servers='localhost:9092',value_serializer=json_serializer)

In [16]:
admin.describe_topics([topic])

[{'error_code': 17,
  'topic': '<kafka.admin.new_topic.NewTopic object at 0x7f3bf3c821d0>',
  'is_internal': False,
  'partitions': []}]

# Sending the Data to the Consumer

In [17]:
for index,row in df.iterrows():
    producer.send("topic3", row.to_json())
    